In [1]:
%run base_functions.ipynb

In [2]:
default_val = None
lm_prediction_file='20210110/lm_pred_lm_pred_nifty_next_50.csv'
lg_prediction_file='20210110/lg_pred_lm_pred_nifty_next_50.csv'

my_interested_columns = ['i_symbol',
 'pr_pChange',
 'm_series',
 'pr_pBand',
 'pr_lastPrice',
 'pr_open',
 'pr_previousClose',
 'pr_close',
 'i_companyName',
 'i_industry',
 'pr_intraDayHighLow_min',
 'pr_intraDayHighLow_max',
 'pr_intraDayHighLow_value',
 'load_time']

In [3]:
# To know more columns
#pd.DataFrame(get_latest_data(['IRCTC'],False)).columns

In [4]:
# file format must be SYMBOL,QTY,INVESTMENT,AVG
port_predict = load_portfolio_file(lm_prediction_file,False) 
up_port_predict = load_portfolio_file(lg_prediction_file,False)  
raw_port = port_predict[['SYMBOL']].drop_duplicates()

In [5]:
%%capture
portfolio_df = populate_nse_data(raw_port,True,interested_columns = my_interested_columns)

In [6]:
#ticker_data = ticker_filter(port_predict,'IRCTC')
#display(ticker_data.pivot(index=['SYMBOL','time_frame'], columns='column', values='value'))
#order_time_dims(ticker_data,'time_frame').pivot(index=['SYMBOL','time_frame'], columns='column', values='value')

In [7]:
prediction_columns = ['f_o', 'f_h','f_l', 'f_c']
reference_columns = ['o', 'h', 'l', 'c', 'v']
columns = ['o','h','l','c']
range_key = 'Data Range'
# Price Band, Delivery %, Company Revenue (Quaterly), P/L (Quaterly),

In [8]:
pivot_df = pd.DataFrame({})

for ticker in portfolio_df['SYMBOL']:
    actual_val = ticker_filter(portfolio_df,ticker)
    #display(actual_val)
    predicted_pivot = order_time_dims(ticker_filter(port_predict,ticker),'time_frame').pivot(index=['SYMBOL','time_frame'], columns='column', values='value')
    
    predicted_pivot['actual_o'] = actual_val['pr_open'][0]
    predicted_pivot['open_diff %'] = (predicted_pivot['actual_o']-predicted_pivot['f_o'])*100/predicted_pivot['actual_o']
    
    predicted_pivot['current_value'] = actual_val['pr_intraDayHighLow_value'][0]
    predicted_pivot.insert(3,'actual_l',actual_val['pr_intraDayHighLow_min'][0])
    predicted_pivot.insert(4,'hit_low', predicted_pivot['actual_l'] < predicted_pivot['f_l'])
    predicted_pivot.insert(2,'actual_h', actual_val['pr_intraDayHighLow_max'][0])
    predicted_pivot.insert(3,'hit_high',predicted_pivot['actual_h'] > predicted_pivot['f_h'])
    predicted_pivot.insert(1,'actual_c',actual_val['pr_lastPrice'][0])
    predicted_pivot.insert(2,'close_diff %',(predicted_pivot['actual_c']-predicted_pivot['f_c'])*100/predicted_pivot['actual_c'])
    predicted_pivot['prev_close'] = actual_val['pr_previousClose'][0]
    #predicted_pivot['f_up'] = predicted_pivot['f_c'] > predicted_pivot['f_o']
    predicted_pivot['actual_up'] = predicted_pivot['actual_c'] > predicted_pivot['actual_o']
    pivot_df = pivot_df.append(predicted_pivot)



In [9]:
up_df = order_time_dims(up_port_predict,'time_frame').pivot(index=['SYMBOL','time_frame'], columns='column', values='value')
up_df['f_up'] = up_df['f_UP'].apply(lambda x: x==1)
pivot_df = pivot_df.join(up_df['f_up'])

In [10]:
time_dims

{'1 week': <function __main__.<lambda>(x)>,
 '1 month': <function __main__.<lambda>(x)>,
 '6 month': <function __main__.<lambda>(x)>,
 '1 year': <function __main__.<lambda>(x)>,
 '5 year': <function __main__.<lambda>(x)>}

In [12]:
pivot_df['is_up_correct'] = pivot_df['f_up'] == pivot_df['actual_up']
display_full(pivot_df)
display_full(apply_filter(pivot_df,'is_up_correct',True).set_index('SYMBOL','time_frame')[['time_frame','f_up','actual_up','is_up_correct']])

f_c  actual_c  close_diff %       f_h  actual_h  \
SYMBOL     time_frame                                                         
NAUKRI     1 week      5,404.79  5,417.30          0.23  5,472.38  5,477.50   
           1 month     5,432.11  5,417.30         -0.27  5,539.25  5,477.50   
           6 month     5,412.92  5,417.30          0.08  5,521.61  5,477.50   
           1 year      5,458.40  5,417.30         -0.76  5,530.88  5,477.50   
           5 year      5,428.49  5,417.30         -0.21  5,535.52  5,477.50   
LTI        1 week      3,921.54  4,088.00          4.07  4,130.58  4,166.60   
           1 month     3,902.91  4,088.00          4.53  4,043.99  4,166.60   
           6 month     4,108.80  4,088.00         -0.51  4,212.24  4,166.60   
           1 year      4,158.49  4,088.00         -1.72  4,239.42  4,166.60   
           5 year      4,107.80  4,088.00         -0.48  4,198.96  4,166.60   
INDUSTOWER 1 week        271.23    265.65         -2.10    273.42    271.90   
           1 month       274.49    265.65         -3.33    276.33    271.90   
           6 month       267.82    265.65         -0.82    275.64    271.90   
           1 year        268.18    265.65         -0.95    274.40    271.90   
           5 year        270.76    265.65         -1.92    275.45    271.90   
HINDZINC   1 week        306.89    291.85         -5.15    328.99    301.00   
           1 month       278.99    291.85          4.41    304.27    301.00   
           6 month       299.05    291.85         -2.47    313.55    301.00   
           1 year        300.13    291.85         -2.84    310.76    301.00   
           5 year        303.36    291.85         -3.94    311.17    301.00   
DMART      1 week      2,963.62  3,057.05          3.06  3,005.50  3,137.00   
           1 month     3,009.88  3,057.05          1.54  3,085.94  3,137.00   
           6 month     2,993.87  3,057.05          2.07  3,043.83  3,137.00   
           1 year      2,968.09  3,057.05          2.91  3,025.81  3,137.00   
           5 year      2,972.11  3,057.05          2.78  3,015.70  3,137.00   
MUTHOOTFIN 1 week      1,279.16  1,261.75         -1.38  1,300.13  1,280.00   
           1 month     1,279.40  1,261.75         -1.40  1,302.45  1,280.00   
           6 month     1,276.24  1,261.75         -1.15  1,310.78  1,280.00   
           1 year      1,288.89  1,261.75         -2.15  1,308.59  1,280.00   
           5 year      1,285.85  1,261.75         -1.91  1,307.42  1,280.00   
SBICARD    1 week        972.52    949.80         -2.39    987.76    970.45   
           1 month       944.64    949.80          0.54    971.20    970.45   
           6 month       959.52    949.80         -1.02    974.74    970.45   
           1 year        962.86    949.80         -1.37    975.73    970.45   
           5 year        962.86    949.80         -1.37    975.73    970.45   
HDFCAMC    1 week      3,101.85  3,183.95          2.58  3,142.38  3,195.95   
           1 month     3,167.05  3,183.95          0.53  3,180.04  3,195.95   
           6 month     3,165.44  3,183.95          0.58  3,201.74  3,195.95   
           1 year      3,146.74  3,183.95          1.17  3,189.54  3,195.95   
           5 year      3,142.96  3,183.95          1.29  3,191.71  3,195.95   
IGL        1 week        555.45    553.95         -0.27    554.55    557.80   
           1 month       552.86    553.95          0.20    560.58    557.80   
           6 month       554.53    553.95         -0.11    563.21    557.80   
           1 year        549.64    553.95          0.78    559.12    557.80   
           5 year        549.99    553.95          0.71    559.21    557.80   
BERGEPAINT 1 week        829.20    819.60         -1.17    827.18    819.60   
           1 month       817.25    819.60          0.29    820.56    819.60   
           6 month       808.84    819.60          1.31    818.20    819.60   
           1 year        817.16    819.60          0.30    821.60    819.60   

,time_frame,f_up,actual_up,is_up_correct
SYMBOL,,,,
NAUKRI,1 month,False,False,True
NAUKRI,6 month,False,False,True
NAUKRI,1 year,False,False,True
NAUKRI,5 year,False,False,True
LTI,6 month,False,False,True
INDUSTOWER,6 month,False,False,True
INDUSTOWER,1 year,False,False,True
INDUSTOWER,5 year,False,False,True
HINDZINC,1 week,False,False,True
